# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 1:24PM,252848,19,ACG-2102492119-7332995,ACG North America LLC,Released
1,Dec 12 2022 1:22PM,252846,10,Stone-8893766,Acute Outpatient Solutions,Released
2,Dec 12 2022 1:13PM,252844,20,8881300,Else Nutrition,Executing
3,Dec 12 2022 1:06PM,252843,10,8893761,"Senseonics, Incorporated",Released
4,Dec 12 2022 1:01PM,252842,19,60018106,"GUSA Granules USA, Inc.",Released
5,Dec 12 2022 12:25PM,252839,12,HH-38735,Hush Hush,Released
6,Dec 12 2022 12:24PM,252838,21,635908,"NBTY Global, Inc.",Released
7,Dec 12 2022 12:23PM,252837,21,418543289,"NBTY Global, Inc.",Released
8,Dec 12 2022 12:14PM,252834,10,0086242014,ISDIN Corporation,Released
9,Dec 12 2022 12:14PM,252834,10,0086242012,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,252842,Released,1
30,252843,Released,1
31,252844,Executing,1
32,252846,Released,1
33,252848,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252842,NaN,1.0
252843,NaN,1.0
252844,1.0,NaN
252846,NaN,1.0
252848,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252778,0.0,1.0
252783,1.0,0.0
252784,0.0,1.0
252785,0.0,8.0
252789,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252778,0,1
252783,1,0
252784,0,1
252785,0,8
252789,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252778,0,1
1,252783,1,0
2,252784,0,1
3,252785,0,8
4,252789,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252778,,1
1,252783,1,
2,252784,,1
3,252785,,8
4,252789,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated"
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc."
5,Dec 12 2022 12:25PM,252839,12,Hush Hush
6,Dec 12 2022 12:24PM,252838,21,"NBTY Global, Inc."
7,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc."
8,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation
45,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC,,1
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions,,1
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition,1,
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated",,1
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc.",,1
5,Dec 12 2022 12:25PM,252839,12,Hush Hush,,1
6,Dec 12 2022 12:24PM,252838,21,"NBTY Global, Inc.",,1
7,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",,1
8,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,,37
9,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",22,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC,1,
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions,1,
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition,,1
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated",1,
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc.",1,
5,Dec 12 2022 12:25PM,252839,12,Hush Hush,1,
6,Dec 12 2022 12:24PM,252838,21,"NBTY Global, Inc.",1,
7,Dec 12 2022 12:23PM,252837,21,"NBTY Global, Inc.",1,
8,Dec 12 2022 12:14PM,252834,10,ISDIN Corporation,37,
9,Dec 12 2022 12:10PM,252833,19,"Innogenix, LLC",1,22


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC,1,
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions,1,
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition,,1
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated",1,
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC,1.0,NaN
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions,1.0,NaN
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition,NaN,1.0
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated",1.0,NaN
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 1:24PM,252848,19,ACG North America LLC,1.0,0.0
1,Dec 12 2022 1:22PM,252846,10,Acute Outpatient Solutions,1.0,0.0
2,Dec 12 2022 1:13PM,252844,20,Else Nutrition,0.0,1.0
3,Dec 12 2022 1:06PM,252843,10,"Senseonics, Incorporated",1.0,0.0
4,Dec 12 2022 1:01PM,252842,19,"GUSA Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3539413,150.0,1.0
12,1011240,3.0,1.0
16,252836,1.0,0.0
19,2022585,8.0,22.0
20,505622,1.0,1.0
21,1011322,4.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3539413,150.0,1.0
1,12,1011240,3.0,1.0
2,16,252836,1.0,0.0
3,19,2022585,8.0,22.0
4,20,505622,1.0,1.0
5,21,1011322,4.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,150.0,1.0
1,12,3.0,1.0
2,16,1.0,0.0
3,19,8.0,22.0
4,20,1.0,1.0
5,21,4.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,150.0
1,12,Released,3.0
2,16,Released,1.0
3,19,Released,8.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,1.0,1.0,0.0,22.0,1.0,0.0
Released,150.0,3.0,1.0,8.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,1.0,1.0,0.0,22.0,1.0,0.0
1,Released,150.0,3.0,1.0,8.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,1.0,1.0,0.0,22.0,1.0,0.0
1,Released,150.0,3.0,1.0,8.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()